In [ ]:
import re
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Chromium browser and WebDriver paths
chromium_binary_path = 'chrome-win64\\chrome.exe'
chromium_driver_path = 'chromedriver-win64\\chromedriver.exe'

# Selenium WebDriver configuration
service = Service(executable_path=chromium_driver_path)
options = webdriver.ChromeOptions()
options.binary_location = chromium_binary_path

options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-features=AutoUpdate')
options.add_argument('--allow-running-insecure-content')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

In [ ]:
# Input CSV file name
input_file_name = 'tiktok_data.csv'

# List of hashtags to collect
hashtags = ["homestay"]

# Function to scroll the page to load more videos
def scroll_page(driver, scroll_count=1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Open the CSV file for writing, including the header
with open(input_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Video URL', 'Title', 'Hashtags'])  # Write header and add search hashtag column

    # Loop through each hashtag
    for hashtag in hashtags:
        print(f"Collecting data for #{hashtag}...")

        # Open the browser and navigate to the TikTok hashtag page
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(f"https://www.tiktok.com/tag/{hashtag}")
        time.sleep(5)  # Wait for the page to load

        # Scroll the page to load more videos
        scroll_page(driver)

        # Find all videos on the hashtag page
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/video/"]')

        for element in elements:
            try:
                video_url = element.get_attribute('href')
                full_title = element.find_element(By.CSS_SELECTOR, 'img').get_attribute('alt')
                title = re.split(r'#', full_title, 1)[0].strip()
                hashtags_list = ' '.join(re.findall(r"#\w+", full_title))
                writer.writerow([video_url, title, hashtags_list])  # Write data and searched hashtag
            except Exception as e:
                #print(f"Error processing video: {e} in hashtag #{hashtag}")
                continue

        driver.quit()
        print(f'Finished collecting #{hashtag}')

print(f"Finished collecting all hashtags! Data saved to {input_file_name}")

Đang thu thập dữ liệu cho #homestay...
Lỗi khi xử lý video: Message: no such element: Unable to locate element: {"method":"css selector","selector":"img"}
  (Session info: chrome=136.0.7103.49); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF61A5AA145+76773]
	GetHandleVerifier [0x00007FF61A5AA1A0+76864]
	(No symbol) [0x00007FF61A368F7A]
	(No symbol) [0x00007FF61A3BF496]
	(No symbol) [0x00007FF61A3BF74C]
	(No symbol) [0x00007FF61A3B1E2C]
	(No symbol) [0x00007FF61A3E739F]
	(No symbol) [0x00007FF61A3B1CF6]
	(No symbol) [0x00007FF61A3E7570]
	(No symbol) [0x00007FF61A40F0CF]
	(No symbol) [0x00007FF61A3E7133]
	(No symbol) [0x00007FF61A3B04D1]
	(No symbol) [0x00007FF61A3B1263]
	GetHandleVerifier [0x00007FF61A86A8ED+2962317]
	GetHandleVerifier [0x00007FF61A864EC2+2939234]
	GetHandleVerifier [0x00007FF61A882FF3+3062419]
	GetHandleVerifier [0x00007FF61A5C4B9A

In [62]:
import pandas as pd
# Đọc file CSV và hiển thị dữ liệu
df = pd.read_csv("tiktok_data.csv")
df

,Video URL,Title,Hashtags
0,https://www.tiktok.com/@homnaodidi/video/74530...,Staycation trong chính thành phố mà mình đang ...,#homnaodidi #hanoi #chillhome #homestay #homes...
1,https://www.tiktok.com/@peanut_homestay/video/...,Homestay lãng mạng Sài Gòn chỉ từ 34k/h chia t...,#homestaysaigon #henho #peanuthomestay
2,https://www.tiktok.com/@our.homestay/video/732...,"Địa điểm hẹn hò bí mật của tui và người iu, nơ...",#reviewhomestay #fypシ #viralvideo #xuhuong #xh...
3,https://www.tiktok.com/@kid_2509/video/7412277...,Một chiếc homestay xinh xắn yên bình ẩn mình d...,#homestay #ninhbinh #dulich
4,https://www.tiktok.com/@emhome.dn/video/743340...,"có một tình yêu nhẹ nhàng, một buổi hẹn hò bìn...",#homestaydanang #netflixandchill #reviewhomest...
5,https://www.tiktok.com/@loveboxstaycation/vide...,Hẹn hò riêng tư tại Lovebox,#homestay #loveboxhomestay #homestaysaigon #he...
6,https://www.tiktok.com/@bemycoffee/video/71056...,Homestay ở Ninh Bình 🌸,#homestay #ninhbinh #foryou #fyp
7,https://www.tiktok.com/@_lianhomestay/video/74...,Tôy không cần gì hơn huhu,#reviewhomestayhanoi #lianhomestay #couple #yt...
8,https://www.tiktok.com/@cadehehehe/video/74311...,tìm chỗ riêng tư để khóc? Đó là thứ mà Cade có,#homestay #homestaysaigon #cadesaigon #cade #c...
9,https://www.tiktok.com/@di.date.di/video/73169...,Hướng nội ngại chỗ đông người thì homestay này...,#homestayvinhomessmartcity #homestay #homestay...


In [ ]:
# Input CSV file name
input_file_name = 'tiktok_data.csv'

# Read video URLs from the CSV file
video_data = []
try:
    with open(input_file_name, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        video_data = list(reader)
except FileNotFoundError:
    print(f"Error: File {input_file_name} not found")
    exit()

# Initialize the browser
driver = webdriver.Chrome(service=service, options=options)

# Browse each video URL to collect interaction data
for row in video_data:
    url = row['Video URL']
    print(f"Collecting data from: {url}")
    driver.get(url)
    time.sleep(4)  # Wait 4 seconds for the page to load

    likes = '0'
    comments = '0'
    shares = '0'
    saves = '0'

    try:
        # Get number of likes
        likes_element = driver.find_element(By.CSS_SELECTOR, "strong[data-e2e='like-count']")
        likes = likes_element.text

        # Get number of comments
        comments_element = driver.find_element(By.CSS_SELECTOR, "strong[data-e2e='comment-count']")
        comments = comments_element.text

        # Get number of shares
        shares_element = driver.find_element(By.CSS_SELECTOR, "strong[data-e2e='share-count']")
        shares = shares_element.text

        # Get number of saves
        saves_element = driver.find_element(By.CSS_SELECTOR, "strong[data-e2e='undefined-count']")
        saves = saves_element.text

        # Calculate Engagement Rate
        row['Likes'] = likes
        row['Comments'] = comments
        row['Shares'] = shares
        row['Saves'] = saves

    except Exception as e:
        print(f"Error collecting data from {url}: {e}")
        row['Likes'] = 'Not found'
        row['Comments'] = 'Not found'
        row['Shares'] = 'Not found'
        row['Saves'] = 'Not found'

# Close the browser after collection is complete
driver.quit()
print("Finished collecting interaction data!")

# Write updated data back to the same CSV file
with open(input_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = video_data[0].keys() if video_data else ['Video URL', 'Title', 'Hashtags', 'Views', 'Likes', 'Comments', 'Shares', 'Saves', 'Engagement Rate']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(video_data)

print(f"Interaction data has been added to {input_file_name}")

Đang thu thập dữ liệu từ: https://www.tiktok.com/@homnaodidi/video/7453084715533257992
Đang thu thập dữ liệu từ: https://www.tiktok.com/@peanut_homestay/video/7398151412611321104
Hoàn thành thu thập dữ liệu tương tác!
Dữ liệu tương tác đã được thêm vào tiktok_data.csv


In [10]:
import pandas as pd
# Đọc file CSV và hiển thị dữ liệu
df = pd.read_csv("tiktok_data.csv")
df

,Video URL,Title,Hashtags,Likes,Comments,Shares,Saves
0,https://www.tiktok.com/@homnaodidi/video/74530...,Staycation trong chính thành phố mà mình đang ...,#homnaodidi #hanoi #chillhome #homestay #homes...,3971,80,1894,2639
1,https://www.tiktok.com/@peanut_homestay/video/...,Homestay lãng mạng Sài Gòn chỉ từ 34k/h chia t...,#homestaysaigon #henho #peanuthomestay,5289,125,1622,2019
2,https://www.tiktok.com/@loveboxstaycation/vide...,Hẹn hò riêng tư tại Lovebox,#homestay #loveboxhomestay #homestaysaigon #he...,11K,428,3185,3047
3,https://www.tiktok.com/@kid_2509/video/7412277...,Một chiếc homestay xinh xắn yên bình ẩn mình d...,#homestay #ninhbinh #dulich,114K,1512,17.5K,21.6K
4,https://www.tiktok.com/@cadehehehe/video/74311...,tìm chỗ riêng tư để khóc? Đó là thứ mà Cade có,#homestay #homestaysaigon #cadesaigon #cade #c...,15K,114,1390,2445


In [ ]:
def add_language_parameter_to_csv(file_path):
    updated_data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as infile:
            reader = csv.DictReader(infile)
            fieldnames = reader.fieldnames
            if 'Video URL' not in fieldnames:
                print("Error: CSV file does not contain a 'Video URL' column.")
                return

            for row in reader:
                url = row['Video URL']
                if url:
                    if "?lang=" not in url:
                        row['Video URL'] = url + "?lang=vi-VN"
                    else:
                        row['Video URL'] = url.split('?')[0] + "?lang=vi-VN"
                updated_data.append(row)

        with open(file_path, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(updated_data)

        print(f"Language parameter has been added/updated in the 'Video URL' column of the file: {file_path}")

    except FileNotFoundError:
        print(f"Error: File {file_path} not found")
    except Exception as e:
        print(f"An error occurred: {e}")

# CSV file name (already contains URL and interaction info)
file_path = 'tiktok_data.csv'

add_language_parameter_to_csv(file_path)

Đã thêm/cập nhật tham số ngôn ngữ vào cột 'Video URL' trong file: tiktok_data.csv


In [69]:
import pandas as pd
# Đọc file CSV và hiển thị dữ liệu
df = pd.read_csv("tiktok_data.csv")
df

,Video URL,Title,Hashtags,Likes,Comments,Shares,Saves,Transcript
0,https://www.tiktok.com/@homnaodidi/video/74530...,Staycation trong chính thành phố mà mình đang ...,#homnaodidi #hanoi #chillhome #homestay #homes...,3971,80,1896,2640,[<selenium.webdriver.remote.webelement.WebElem...
1,https://www.tiktok.com/@peanut_homestay/video/...,Homestay lãng mạng Sài Gòn chỉ từ 34k/h chia t...,#homestaysaigon #henho #peanuthomestay,5295,125,1627,2020,[ERROR] Không tìm thấy transcript


In [ ]:
# Input CSV file name
input_file_name = 'tiktok_data.csv'

# Open the CSV and process
updated_data = []
try:
    with open(input_file_name, 'r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames

        if 'Video URL' not in fieldnames:
            print("Error: CSV file does not contain a 'Video URL' column.")
            driver.quit()
            exit()

        if 'Transcript' not in fieldnames:
            fieldnames.append('Transcript')
        
        driver = webdriver.Chrome(service=service, options=options)

        for row in reader:
            url = row['Video URL']
            print(f"Processing video: {url}")
            transcript_text = ''

            driver.get('https://script.tokaudit.io/')
            try:
                # Click bottom-left to close overlay popup
                webdriver.ActionChains(driver).move_by_offset(10, 500).click().perform()
                time.sleep(1)

                # Find input box and paste URL
                insert_link_input = WebDriverWait(driver, 30).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, 'textarea[placeholder="Enter Video Url"]')))

                insert_link_input.click()                
                insert_link_input.send_keys(url)

                # Click START button
                start_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "START")]')))

                start_button.click()
                time.sleep(1)

                # Remove readonly checkbox and check it
                checkbox = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="checkbox"].mr-1'))
                )
                driver.execute_script("arguments[0].removeAttribute('readonly'); arguments[0].click();", checkbox)
                time.sleep(1)

                # Get the transcript text elements
                text_elements = driver.find_elements(By.CSS_SELECTOR, ".text.hover\\:text-gray-200.text-xs.text-justify")

                # Extract and combine the text
                transcript = []
                for element in text_elements:
                    transcript.append(element.text)

                # Join all text pieces
                full_transcript = " ".join(transcript)
                row['Transcript'] = full_transcript

            except Exception as e:
                row['Transcript'] = 'Subtitles Not Available'

            updated_data.append(row)

    # Write back to CSV (or rename if you want to keep the original)
    with open(input_file_name, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(updated_data)

    print(f"Transcript has been updated in file: {input_file_name}")

except FileNotFoundError:
    print(f"File not found: {input_file_name}")
except Exception as e:
    print(f"General error: {e}")
finally:
    driver.quit()
    print("Process completed.")

Đang xử lý video: https://www.tiktok.com/@homnaodidi/video/7453084715533257992?lang=vi-VN
Đang xử lý video: https://www.tiktok.com/@peanut_homestay/video/7398151412611321104?lang=vi-VN
Đã cập nhật transcript vào file: tiktok_data.csv
Hoàn tất quá trình.


In [73]:
import pandas as pd
# Đọc file CSV và hiển thị dữ liệu
df = pd.read_csv("tiktok_data.csv")
df

,Video URL,Title,Hashtags,Likes,Comments,Shares,Saves,Transcript
0,https://www.tiktok.com/@homnaodidi/video/74530...,Staycation trong chính thành phố mà mình đang ...,#homnaodidi #hanoi #chillhome #homestay #homes...,3971,80,1896,2640,staycation trong chính thành phố mà mình đang ...
1,https://www.tiktok.com/@peanut_homestay/video/...,Homestay lãng mạng Sài Gòn chỉ từ 34k/h chia t...,#homestaysaigon #henho #peanuthomestay,5295,125,1627,2020,Subtitles Not Available
